In [3]:
from src.data.tushare.basic.calendar import _calendar
_calendar

,trade,td,pre,cd_index,td_index
calendar,,,,,
19901219,1,19901219,-1,0,0
19901220,1,19901220,19901219,1,1
19901221,1,19901221,19901220,2,2
19901222,0,19901221,19901220,3,2
19901223,0,19901221,19901220,4,2
...,...,...,...,...,...
20241227,1,20241227,20241226,12427,8309
20241228,0,20241227,20241226,12428,8309
20241229,0,20241227,20241226,12429,8309


In [1]:
from src.data.tushare.basic import TradeDate
TradeDate(20241026) , TradeDate(20241026) - 1 , TradeDate(20241026) + 2


(TradeDate(calendar=20241026,trade=20241025),
 TradeDate(calendar=20241024,trade=20241024),
 TradeDate(calendar=20241029,trade=20241029))

In [2]:
int(TradeDate(20241029))


20241029

In [3]:
from src.data.tushare.basic import CALENDAR , TRADE_DATA

In [2]:
'''
import pandas as pd
dates = CALENDAR.trailing(20241029 , 60)
TRADE_DATA.load(dates)

df_list = []
for i in range(len(dates[1:])):
    trd = TRADE_DATA.get_trd(dates[1 + i] , ['secid','pctchange']) 
    val = TRADE_DATA.get_val(dates[0 + i] , ['secid','circ_mv'])
    ret = trd.merge(val , on = 'secid').assign(date = dates[1 + i])
    df_list.append(ret)
df = pd.concat(df_list)
'''

stk_ret = TRADE_DATA.get_rets(20241029 , 20241029)

In [3]:
stk_ret

secid,1,2,4,6,7,8,9,10,11,12,...,873726,873806,873833,920002,920008,920016,920019,920088,920099,920118
date,,,,,,,,,,,,,,,,,,,,,
20241029,-0.008591,-0.027689,-0.045045,0.072432,-0.025388,-0.047059,-0.052372,-0.033898,-0.031008,-0.024476,...,0.039813,-0.024129,-0.049333,0.046296,0.014649,0.019858,-0.012681,3.972678,0.036819,0.059115


In [6]:
CALENDAR.td_within(20241029 , 20241029)


array([20241029])

In [2]:
from src.data.loader import DataVendor , DATAVENDOR
dv = DataVendor()
dv.secid()


array([     1,      2,      3, ..., 920088, 920099, 920118])

In [13]:
date = 20241029
dv.all_stocks[(dv.all_stocks.list_dt <= date) & (dv.all_stocks.delist_dt > date)].sort_values('secid').secid


2097         1
254          2
1209         4
255          6
2552         7
         ...  
5555    920016
5628    920019
5629    920088
5293    920099
5550    920118
Name: secid, Length: 5361, dtype: int32

In [16]:
dv.all_stocks.drop_duplicates('secid')

,index,secid,sec_name,exchange_name,list_dt,delist_dt
2097,2124,1,平安银行,SZSE,19910403,99991231
254,281,2,万科A,SZSE,19910129,99991231
1479,1506,3,PT金田A(退市),SZSE,19910703,20020614
1209,1236,4,国华网安,SZSE,19901201,99991231
1210,1237,5,ST星源(退市),SZSE,19901210,20240426
...,...,...,...,...,...,...
5628,8039,920019,铜冠矿建,BSE,20241011,99991231
5419,7350,920066,N科拜尔,BSE,20241031,99991231
5629,8042,920088,科力股份,BSE,20241029,99991231
5293,6837,920099,瑞华技术,BSE,20240925,99991231


In [10]:
dv.all_stocks.list_dt

2097    19910403
254     19910129
1479    19910703
1209    19901201
1210    19901210
          ...   
5628    20241011
5419    20241031
5629    20241029
5293    20240925
5550    20240822
Name: list_dt, Length: 5652, dtype: int32

In [9]:
import os
import importlib.util
import inspect

from src.basic import PATH
folder_path =  PATH.main.joinpath('src_factors')

classes = []
for level_path in folder_path.iterdir():
    if not level_path.is_dir(): continue
    print(level_path)
    for file_path in level_path.iterdir():
        if file_path.suffix != '.py': continue
        spec_name = f'{level_path.stem}.{file_path.stem}'
        module_name = file_path.stem
        
        spec = importlib.util.spec_from_file_location(spec_name, file_path)
        assert spec is not None and spec.loader is not None , f'{file_path} is not a valid module'
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        
        for name, obj in inspect.getmembers(module, inspect.isclass):
            classes.append((obj.__module__ , obj))
classes

d:\Coding\learndl\learndl\src_factors\level0
d:\Coding\learndl\learndl\src_factors\level1


[('src.factor.classes.factor_calc',
  src.factor.classes.factor_calc.StockFactorCalculator),
 ('src.data.tushare.basic.calendar',
  src.data.tushare.basic.calendar.TradeDate),
 ('level0.ret20_raw', level0.ret20_raw.ret20_raw)]

In [8]:
level_path.stem

'__init__'

In [1]:
import os
import importlib.util
import inspect

def load_classes_from_folder(folder_path):
    classes = []
    
    # 获取文件夹的绝对路径
    folder_path = os.path.abspath(folder_path)
    folder_name = os.path.basename(folder_path)
    
    # 遍历文件夹中的所有文件
    for filename in os.listdir(folder_path):
        if filename.endswith('.py'):
            module_name = filename[:-3]  # 去掉 .py 后缀
            file_path = os.path.join(folder_path, filename)
            
            # 动态导入模块
            spec = importlib.util.spec_from_file_location(f"{folder_name}.{module_name}", file_path)
            module = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(module)
            
            # 获取模块中的所有类
            for name, obj in inspect.getmembers(module, inspect.isclass):
                # 确保类是定义在当前模块中，而不是导入的
                if obj.__module__ == f"{folder_name}.{module_name}":
                    classes.append(obj)
    
    return classes

# 使用示例
folder_path =  './src/factor/factor_algorithm'
all_classes = load_classes_from_folder(folder_path)
print(all_classes)

In [2]:
folder_path =  './src/factor/factor_algorithm'
all_classes = load_classes_from_folder(folder_path)
print(all_classes)

[<class 'factor_algorithm.behavior.ret20_raw'>]


In [3]:
import os
import importlib.util
import inspect
import sys

def load_classes_from_folder(folder_path):
    classes = []
    
    # 将文件夹路径添加到 sys.path
    sys.path.append(folder_path)
    
    # 遍历文件夹中的所有文件
    for filename in os.listdir(folder_path):
        if filename.endswith('.py'):
            module_name = filename[:-3]  # 去掉 .py 后缀
            file_path = os.path.join(folder_path, filename)
            
            # 动态导入模块
            spec = importlib.util.spec_from_file_location(module_name, file_path)
            module = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(module)
            
            # 获取模块中的所有类
            for name, obj in inspect.getmembers(module, inspect.isclass):
                # 确保类是定义在当前模块中，而不是导入的
                if obj.__module__ == module_name:
                    classes.append(obj)
    
    # 从 sys.path 中移除文件夹路径
    sys.path.remove(folder_path)
    
    return classes

In [6]:
from src.factor.factor_algorithm.behavior import ret20_raw

In [5]:
beh.ret20_raw

src.factor.factor_algorithm.definition.behavior.ret20_raw

In [16]:
from pathlib import Path
file_path = Path('./src/factor/factor_algorithm/definition/behavior.py').absolute()

In [19]:
str(file_path)

'd:\\Coding\\learndl\\learndl\\src\\factor\\factor_algorithm\\definition\\behavior.py'

In [20]:
module = importlib.import_module(str(file_path))

ModuleNotFoundError: No module named 'd:\\Coding\\learndl\\learndl\\src\\factor\\factor_algorithm\\definition\\behavior'

In [7]:
import os  
import importlib.util  
import sys  
  
file_path = 'd:\\Coding\\learndl\\learndl\\src_api\\factor_algorithm\\behavior'  
module_name = "behavior"  
  
# 创建模块的规格  
spec = importlib.util.spec_from_file_location(module_name, file_path)  
# 创建一个新的模块  
module = importlib.util.module_from_spec(spec)  
# 执行模块中的代码  
sys.modules[module_name] = module  
spec.loader.exec_module(module)  
  
# 现在你可以使用 module 来访问 behavior.py 中定义的内容

ImportError: attempted relative import with no known parent package

In [7]:
import os
import importlib.util
import inspect
import sys

folder_path = './src/factor/factor_algorithm/definition'
classes = []

# 将文件夹路径添加到 sys.path
sys.path.append(folder_path)

# 遍历文件夹中的所有文件
for filename in os.listdir(folder_path):
    if filename.endswith('.py'):
        module_name = filename[:-3]  # 去掉 .py 后缀
        file_path = os.path.join(folder_path, filename)
        
        # 动态导入模块
        #spec = importlib.util.spec_from_file_location(module_name, file_path)
        #module = importlib.util.module_from_spec(spec)
        #spec.loader.exec_module(module)
        module = importlib.import_module(file_path)
        
        # 获取模块中的所有类
        for name, obj in inspect.getmembers(module, inspect.isclass):
            # 确保类是定义在当前模块中，而不是导入的
            if obj.__module__ == module_name:
                classes.append(obj)

# 从 sys.path 中移除文件夹路径
sys.path.remove(folder_path)


TypeError: the 'package' argument is required to perform a relative import for './src/factor/factor_algorithm/definition\\behavior.py'

In [6]:
module_name

'behavior'

In [5]:
file_path

'./src/factor/factor_algorithm/definition\\behavior.py'

In [3]:
module.__dir__()

['__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__file__',
 '__cached__',
 '__builtins__',
 'np',
 'pd']

In [9]:
from abc import ABC, abstractmethod
from collections.abc import Iterable
from typing import Any , final , Sequence
import numpy as np
import pandas as pd
import datetime

from pathlib import Path
from src.basic import PATH

from src.data.tushare.basic import CALENDAR , TRADE_DATA , TradeDate
from src.data.loader import DATAVENDOR
from src.func.time import today


class StockFactorCalculator(ABC):
    INIT_DATE = 20070101

    @final
    def __init__(self):
        self.assign_attr()
        self.validate_attr()
        self.factors : dict[int , pd.DataFrame] = {}

    def __repr__(self):
        return f'StockFactor(name={self.factor_name},category0={self.category0},category1={self.category1},from={self.init_date})'

    @property
    def factor_name(self):
        return self.__class__.__name__
    
    @property
    def factor_folder(self):
        return Path(f'{PATH.factor}/{self.category0}/{self.category1}/{self.factor_name}')
    
    @property
    def factor_values(self):
        return pd.concat([df.assign(date = d) for d , df in self.factors.items()]).reset_index().set_index(['date','secid'])

    def factor_path(self , date : int | Any , mkdir = True):
        path = self.factor_folder.joinpath(f'{int(date) // 10000}/{self.factor_name}.{int(date)}.feather')
        if mkdir: path.parent.mkdir(parents=True , exist_ok=True)
        return path
    
    @abstractmethod
    def assign_attr(self) -> None:
        '''set attribute of factor_data'''
        self.init_date : int = -1
        self.category0 : str = '' # 'fundamental' , 'analyst' , 'high_frequency' , 'behavior' , 'money_flow' , 'alternative'
        self.category1 : str = ''
        self.description : str = ''
    
    @abstractmethod
    def calc_factor(self , date : int) -> pd.DataFrame:
        '''calculate factor value , must have secid and factor_value / factor_name columns'''
        return pd.DataFrame()
    
    @final
    def validate_attr(self):
        '''validate attribute of factor'''
        assert self.init_date >= self.INIT_DATE , f'init_date should be later than {self.INIT_DATE} , but got {self.init_date}'

        assert self.category0 in ['fundamental' , 'analyst' , 'high_frequency' , 'behavior' , 'money_flow' , 'alternative'] , \
            f'category0 is should be in [fundamental , analyst , high_frequency , behavior , money_flow , alternative], but got {self.category0}'
        
        category1_list = {
            'fundamental' : ['quality' , 'growth' , 'value' , 'earning'] ,
            'analyst' : ['surprise' , 'coverage' , 'forecast' , 'adjustment'] ,
            'high_frequency' : ['hf_momentum' , 'hf_volatility' , 'hf_correlation' , 'hf_liquidity'] ,
            'behavior' : ['momentum' , 'volatility' , 'correlation' , 'liquidity'] ,
            'money_flow' : ['holding' , 'trading'] ,
            'alternative' : None
        }[self.category0]
        
        if category1_list is not None:
            assert self.category1 in category1_list , \
                f'category1 is should be in {category1_list}, but got {self.category1}'
        else:
            assert self.category1 , 'category1 is not set'
        
        assert self.description , 'description is not set'

        return self
    
    @final
    def validate_value(self , date : int , df : pd.DataFrame , strict = False):
        '''validate factor value'''

        assert 20991231 >= date >= self.init_date , \
            f'calc_date is should be in [{self.init_date} , 20991231], but got {date}'

        mininum_finite_count = 100 if strict else 0
        actual_finite_count = np.isfinite(df[self.factor_name].to_numpy()).sum()
        if actual_finite_count < mininum_finite_count:
            raise ValueError(f'factor_value must have at least {mininum_finite_count} finite values , but got {actual_finite_count}')
        
        return self

    def calculate(self , date : int | Iterable | Any , overwrite = False):
        '''calculate factor value of a given date and store to factor_data'''
        if isinstance(date , Iterable):
            for d in date: self.calculate(d , overwrite)
        else:
            date = int(date)
            assert date >= self.init_date , f'date is should be greater than or equal to {self.init_date}, but got {date}'
            if overwrite or date not in self.factors: 
                df = self.calc_factor(date).reset_index().set_index('secid').\
                    rename(columns={'factor_value':self.factor_name})[[self.factor_name]]
                self.factors[date] = df
        return self

    def deploy(self , strict = True , overwrite = False , show_progress = False):
        '''store factor data after calculate'''
        dates = list(self.factors.keys())
        for date in dates:
            df = self.factors.pop(date)
            path = self.factor_path(date , True)
            if path.exists() and not overwrite: 
                if show_progress: print(f'Factor:{self.factor_name} at date {date} already there')
                continue
            try:
                self.validate_value(date , df , strict = strict)
                df.to_feather(self.factor_path(date , True))
                if show_progress: print(f'Factor:{self.factor_name} at date {date} deploy successful')
            except ValueError as e:
                print(f'Factor:{self.factor_name} at date {date} is invalid: {e}')

        return self

    def load(self, date : int | Iterable | Any):
        '''load factor data from storage'''
        if isinstance(date , Iterable):
            dfs = {int(d):self.load(d) for d in date}
            dfs = [df.assign(date = d) for d , df in dfs.items() if isinstance(df , pd.DataFrame)]
            if dfs:
                return pd.concat(dfs).reset_index().set_index(['date','secid'])
            else:
                return None
        else:
            if date in self.factors: return self.factors[int(date)]
            factor_path = self.factor_path(date = date)
            if factor_path.exists():
                return pd.read_feather(factor_path)
            else:
                return None

    def stored_dates(self):
        paths = PATH.list_files(self.factor_folder , recur=True)
        dates = np.array(sorted(PATH.R_path_date(paths)) , dtype=int)
        return dates

    def Update(self , start : int = -1 , end : int = 99991231):
        '''update factor data from self.init_date to today'''
        store_dates = self.stored_dates()
        dates = np.setdiff1d(CALENDAR.td_within(max(start , self.init_date) , end) , store_dates)

        if len(dates):
            for date in dates: 
                self.calculate(date).deploy()
            print(f'Factor:{self.factor_name} update finish : {dates[0]} - {dates[-1]}')

# subclass of StockFactorCalculator
class ret20_raw(StockFactorCalculator):
    def assign_attr(self):
        self.init_date = 20070101
        self.category0 = 'behavior'
        self.category1 = 'momentum'
        self.description = 'raw return of 20 days'

    def calc_factor(self , date : int):
        ret = TRADE_DATA.get_rets(TradeDate(date) - 19 , date)
        df = (np.exp(np.log(ret + 1).sum(axis = 0)) - 1).rename('factor_value').to_frame()
        return df

f = ret20_raw()
f.calculate([20241025,20241029]).deploy(show_progress=True)


Factor:ret20_raw at date 20241025 already there
Factor:ret20_raw at date 20241029 already there


StockFactor(name=ret20_raw,category0=behavior,category1=momentum,from=20070101)

In [12]:
f.load([20241024,20241025])

ret20_raw
date     secid            
20241024 1        0.223953
         2        0.347119
         4        0.683492
         6        0.873834
         7        0.250922
...                    ...
20241025 920008   1.628814
         920016   0.550515
         920019   5.254042
         920099   0.763158
         920118   0.545724

[10715 rows x 1 columns]

In [8]:
f.stored_dates()

array([20241024, 20241025, 20241028, 20241029, 20241030])

In [9]:
a = {'a' : 1 , 'b' : 2}
keys = list(a.keys())

[print(a.pop(k)) for k in keys]

a

1
2


{}

In [3]:
f.factor_values

ret20_raw
date     secid            
20241029 1        0.137164
         2        0.282302
         4        0.303810
         6        1.259685
         7        0.245044
...                    ...
         920016   0.515279
         920019   5.473437
         920088   3.972678
         920099   0.852632
         920118   0.664867

[5360 rows x 1 columns]

In [5]:
from src.data.tushare.basic import TRADE_DATA , TradeDate
date = 20241029
ret = TRADE_DATA.get_rets(int(TradeDate(date) - 19) , date)



NameError: name 'np' is not defined

In [26]:
import numpy as np
import pandas as pd

factor_name = 'try'
df = (np.exp(np.log(ret + 1).sum(axis = 0)) - 1).rename('factor_value').to_frame().reset_index().set_index('secid').\
     rename(columns={'factor_value':factor_name})[[factor_name]]
df


,try
secid,
1,0.137164
2,0.282302
4,0.303810
6,1.259685
7,0.245044
...,...
920016,0.515279
920019,5.473437
920088,3.972678


In [24]:
df

,factor_value
secid,
1,0.137164
2,0.282302
4,0.303810
6,1.259685
7,0.245044
...,...
920016,0.515279
920019,5.473437
920088,3.972678
